- Dict with top models for each horizon
- Trainflow where number of runs equals horizon
- Append each training day to master list for performance tracking?
- Pull in performance over last n_days (120 min?, 30 and 10?)
    - Focused on precision
- Also ensemble scoring

In [2]:
import performance_flow
import importlib
importlib.reload(performance_flow)
import warnings
warnings.filterwarnings("ignore", category=RuntimeWarning)
warnings.filterwarnings("ignore", category=UserWarning)
import train_flow
importlib.reload(train_flow)
import pandas as pd
import numpy as np
from xgboost import XGBClassifier

# --------- #
# LEAVE ME  #
# --------- #
ticker = 'QQQ'
include_minute_feats = "N"
returns = [1, 2, 3, 5, 10, 20, 30]
df_daily, feature_sets, return_cols, daily_cols, feature_dict, features = train_flow.import_data(ticker, include_minute_feats, returns)

# ---------- #
# UPDATE ME  #
# ---------- #
# Add any new features
#df_daily[[f"{c}_sum10" for c in df_daily.columns if c.startswith("Past_Return_")]] = (df_daily.sort_values(by="Date", ascending=True).filter(like="Past_Return_").rolling(10, min_periods=1).sum())
#past_ret_cols = [c for c in df_daily.columns if c.startswith("Past_Return%") or c.endswith("sum10")]
#past_perc_cols = [c for c in df_daily.columns if c.startswith("Past_Return%")]
#past_sum_cols = [c for c in df_daily.columns if c.endswith("sum10")]
#cutoff_date = '2026-02-06' # Unlessretraining all baseline models, leave alone

# --------- #
# LEAVE ME  #
# --------- #
#df_main = df_daily[df_daily['Date'] <= cutoff_date].copy() #cutoff date for baseline performance metrics

Available Feature Sets: dict_keys(['ma', 'rsi', 'macd', 'volume', 'atr_adx', 'volatility', 'vix_skew', 'experimental_slope', 'past_return'])


# Parse Top Models per Horizon

In [32]:
import importlib
importlib.reload(performance_flow)
# ---------- #
# UPDATE ME  #
# ---------- #
min_th = 0.55
cov_th = 0.75
horizons = [10]#[2, 5, 10, 20, 30]

for r in horizons:
    
    results_file_name = f"h{r}_baseline_v2.csv" # Match prior cell saved as file name horizon_2_baseline_new
    return_cols, perf_df = performance_flow.import_data(results_file_name, df_daily)
    composite_score = performance_flow.run_performance(perf_df[perf_df['horizon'] == r], min_th, cov_th)
    bucket_df = performance_flow.bucket_scores(df_daily, perf_df[perf_df['horizon'] == r], returns, min_th)

    top_10_r = (
    composite_score.sort_values("composite", ascending=False)
    .drop_duplicates(subset=["features"], keep="first").head(10).copy())

    keys = ["horizon", "features", "train_years", "min_feats", "pi_size", "pi_handling", "model"]

    # Ensure dtypes match so the join actually hits
    for df in (top_10_r, perf_df):

        df["horizon"] = r
        df["features"] = df["features"].astype(str)
        df["model"]       = df["model"].astype(str)
        df["pi_size"]     = df["pi_size"]
        df["pi_handling"]     = df["pi_handling"].astype(str)
        df["train_years"] = df["train_years"].astype(int)
        df["min_feats"]   = df["min_feats"].astype(int)

    # Filter master predictions to only rows matching one of the 10 configs
    pred_filtered = perf_df.merge(top_10_r[keys].drop_duplicates(), on=keys, how="inner")
    #print(len(pred_filtered))
    pred_filtered.to_csv(f"h{r}_top10_raw.csv", index=False)

In [34]:
pred_filtered

,model,test_days,pred,acc,test_n,test_pos_n,train_n,test_start,test_end,train_years,...,features,pi_handling,run,test_neg_n,test_pos_frac,test_neg_frac,train_start,train_end,n_features,Date
0,xgboost-3,1,0.95,0.0,1,0,980,2026-01-21,2026-01-21,4,...,ma_rel,include_new,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2026-01-21
1,xgboost-3,1,1.00,1.0,1,1,980,2026-01-20,2026-01-20,4,...,ma_rel,include_new,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2026-01-20
2,xgboost-3,1,0.95,1.0,1,1,980,2026-01-16,2026-01-16,4,...,ma_rel,include_new,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2026-01-16
3,xgboost-3,1,0.90,1.0,1,1,980,2026-01-15,2026-01-15,4,...,ma_rel,include_new,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2026-01-15
4,xgboost-3,1,1.00,1.0,1,1,980,2026-01-14,2026-01-14,4,...,ma_rel,include_new,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2026-01-14
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1200,xgboost-3,1,1.00,0.0,1,0,1225,2026-01-30,2026-01-30,5,...,ma_lag-past_return,run_separately,1.0,1.0,0.0,1.0,2021-03-02,2026-01-14,8.0,2026-01-30
1201,xgboost-3,1,0.25,1.0,1,0,1225,2026-01-29,2026-01-29,5,...,ma_lag-past_return,run_separately,2.0,1.0,0.0,1.0,2021-03-01,2026-01-13,8.0,2026-01-29
1202,xgboost-3,1,0.10,1.0,1,0,1225,2026-01-28,2026-01-28,5,...,ma_lag-past_return,run_separately,3.0,1.0,0.0,1.0,2021-02-26,2026-01-12,8.0,2026-01-28
1203,xgboost-3,1,0.50,0.0,1,0,1225,2026-01-27,2026-01-27,5,...,ma_lag-past_return,run_separately,4.0,1.0,0.0,1.0,2021-02-25,2026-01-09,8.0,2026-01-27


In [35]:
from itertools import chain
import deployment_flow
import importlib
importlib.reload(deployment_flow)

def resolve_feature_cols(feature_set_name: str, features_dict: dict, sep: str = "-") -> list[str]:

    # --- Kitchen sink case ---
    if feature_set_name == "kitch_sink":
        all_cols = chain.from_iterable(features_dict.values())
        # dedupe preserve order
        seen = set()
        out = []
        for c in all_cols:
            if c not in seen:
                seen.add(c)
                out.append(c)
        return out

    # --- Normal composite case ---
    parts = feature_set_name.split(sep)

    cols = []
    for p in parts:
        if p not in features_dict:
            raise KeyError(f"{p} not in features_dict")
        cols.append(features_dict[p])

    # flatten + dedupe
    seen = set()
    out = []
    for c in chain.from_iterable(cols):
        if c not in seen:
            seen.add(c)
            out.append(c)

    return out

h=[10]

for r in h:
    
    df = pd.read_csv(f"h{r}_top10_raw.csv")

    df["feature_cols"] = df["features"].apply(lambda x: resolve_feature_cols(x, feature_dict))

    grain_cols = ["horizon","features","train_years","min_feats","pi_size","model","pi_handling"]

    max_train = (
        df.groupby(grain_cols, as_index=False)["test_start"]
        .max()
        .rename(columns={"test_start": "max_test_start"})
    )

    top_10_unique = (
        df[grain_cols].drop_duplicates(subset=grain_cols, keep="first")
        .merge(df[grain_cols + ["feature_cols"]].drop_duplicates(subset=grain_cols), on=grain_cols, how="left")
        .merge(max_train, on=grain_cols, how="left")   # <-- this is the missing piece
    )
    
    for row in top_10_unique.itertuples(index=False):

        target_horizon = row.horizon
        pi_handling    = 'run_separately' #row.pi_handling
        feature_cols   = row.feature_cols   # list-of-cols wrapped in a list
        list_name      = row.features
        train_year     = row.train_years
        pi_year        = row.pi_size
        min_feat       = row.min_feats
        max_test_start = row.max_test_start
        days_assessed  = len(df_daily.iloc[r:][df_daily['Date'] > max_test_start].copy())
        groups = list_name.split("-")

        model = XGBClassifier(n_estimators=300, random_state=42, n_jobs=-1)
        model_name = "xgboost-3"

        print(f"{target_horizon} | {pi_handling} | {list_name} | {train_year} | {pi_year} | {min_feat} | {days_assessed}")
        results_df = deployment_flow.run_deploy_flow(days_assessed, r, pi_handling, feature_cols, df_daily, model_name, model,
                        train_year, pi_year, min_feat, list_name, feature_dict, groups)

        performance_df = pd.read_csv(f"h{r}_baseline.csv")
        df_concat = pd.concat([performance_df, results_df], ignore_index=True)    
        #df_concat.to_csv(f"h{r}_baseline_v2.csv", index=False)

10 | run_separately | ma_rel | 4 | 1.5 | 6 | 7
Running for horizon 10 | run_separately
ma_rel: 20 | 6 | ['Close_Rel_Min100', 'Close_Rel_Min200', 'Close_Rel_Min25', 'Rel_Min_120', 'Rel_Min_240', 'Rel_Min_60']
Run 1/7 | Train: 2022-02-17 → 2026-01-14 | Test: 2026-01-30 → 2026-01-30 | Train_n=980 | Test_n=1 | (PI Years: 1.5 - Feats: 6)
ma_rel: 20 | 7 | ['Close_Rel_Min100', 'Close_Rel_Min200', 'Close_Rel_Min25', 'Rel_Max_30', 'Rel_Min_120', 'Rel_Min_240', 'Rel_Min_60']
Run 2/7 | Train: 2022-02-16 → 2026-01-13 | Test: 2026-01-29 → 2026-01-29 | Train_n=980 | Test_n=1 | (PI Years: 1.5 - Feats: 6)
ma_rel: 20 | 6 | ['Close_Rel_Min100', 'Close_Rel_Min200', 'Close_Rel_Min25', 'Rel_Min_120', 'Rel_Min_240', 'Rel_Min_60']
Run 3/7 | Train: 2022-02-15 → 2026-01-12 | Test: 2026-01-28 → 2026-01-28 | Train_n=980 | Test_n=1 | (PI Years: 1.5 - Feats: 6)
ma_rel: 20 | 6 | ['Close_Rel_Min100', 'Close_Rel_Min200', 'Close_Rel_Min25', 'Rel_Min_120', 'Rel_Min_240', 'Rel_Min_60']
Run 4/7 | Train: 2022-02-14 → 2026

In [37]:
results_df

,features,horizon


In [36]:
df_concat

,model,test_days,pred,acc,test_n,test_pos_n,train_n,test_start,test_end,train_years,horizon,pi_size,min_feats,features,pi_handling
0,xgboost-3,1.0,0.40,1.0,1.0,0.0,1470.0,2026-01-21,2026-01-21,6.0,10,1.5,8.0,ma_lag,include_new
1,xgboost-3,1.0,0.60,1.0,1.0,1.0,1470.0,2026-01-20,2026-01-20,6.0,10,1.5,8.0,ma_lag,include_new
2,xgboost-3,1.0,0.45,0.0,1.0,1.0,1470.0,2026-01-16,2026-01-16,6.0,10,1.5,8.0,ma_lag,include_new
3,xgboost-3,1.0,0.30,0.0,1.0,1.0,1470.0,2026-01-15,2026-01-15,6.0,10,1.5,8.0,ma_lag,include_new
4,xgboost-3,1.0,0.10,0.0,1.0,1.0,1470.0,2026-01-14,2026-01-14,6.0,10,1.5,8.0,ma_lag,include_new
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12835,xgboost-3,1.0,0.40,1.0,1.0,0.0,1225.0,2025-08-08,2025-08-08,5.0,10,1.5,4.0,experimental_slope-past_return,run_separately
12836,xgboost-3,1.0,1.00,0.0,1.0,0.0,1225.0,2025-08-07,2025-08-07,5.0,10,1.5,4.0,experimental_slope-past_return,run_separately
12837,xgboost-3,1.0,1.00,0.0,1.0,0.0,1225.0,2025-08-06,2025-08-06,5.0,10,1.5,4.0,experimental_slope-past_return,run_separately
12838,xgboost-3,1.0,0.80,1.0,1.0,1.0,1225.0,2025-08-05,2025-08-05,5.0,10,1.5,4.0,experimental_slope-past_return,run_separately


In [272]:


grain_cols = ["horizon", "features", "train_years", "min_feats", "pi_size", "model", "pi_handling", "test_start"]
top_10_unique = (df[grain_cols].sort_values(by='test_start', ascending=True).drop_duplicates(subset=grain_cols, keep="first").copy())

top_10_unique = (
    top_10_unique.merge(
        df[grain_cols + ["feature_cols"]].drop_duplicates(subset=grain_cols),
        on=grain_cols,
        how="left"
    )
)
r = 2
for row in top_10_unique.itertuples(index=False):

    target_horizon = row.horizon
    pi_handling    = 'include_new' #row.pi_handling
    feature_cols   = row.feature_cols   # list-of-cols wrapped in a list
    list_name      = row.features
    train_year     = row.train_years
    pi_year        = row.pi_size
    min_feat       = 8 #row.min_feats
    max_train_date = row.test_start
    days_assessed  = len(df_daily.iloc[r:][df_daily['Date'] > max_train_date].copy())

model = XGBClassifier(n_estimators=300, random_state=42, n_jobs=-1)
model_name = "xgboost-3"

#print(f"{target_horizon} | {pi_handling} | {list_name} | {train_year} | {pi_year} | {min_feat} | {days_assessed} | {feature_cols}")
results_df = deployment_flow.run_deploy_flow(days_assessed, r, pi_handling, feature_cols, df_daily, model_name, model,
                   train_year, pi_year, min_feat, list_name)

results_df

Running for horizon 2 | include_new
71 | 8 | All Cols: ['Close_slope10', 'SMA_10_Lag100_min', 'SMA_10_Lag10_max', 'SMA_10_Lag200_min', 'Williams_%R_5', 'Zscore_10', 'Zscore_5', 'Zscore_50']
Run 1/5 | Train: 2021-03-24 → 2026-02-06 | Test: 2026-02-11 → 2026-02-11 | Train_n=1225 | Test_n=1 | (PI Years: 1.5 - Feats: 8)
71 | 8 | All Cols: ['CCI_14', 'Close_slope10', 'Close_slope25', 'SMA_10_Lag200_min', 'Williams_%R_5', 'Zscore_10', 'Zscore_5', 'Zscore_50']
Run 2/5 | Train: 2021-03-23 → 2026-02-05 | Test: 2026-02-10 → 2026-02-10 | Train_n=1225 | Test_n=1 | (PI Years: 1.5 - Feats: 8)
71 | 8 | All Cols: ['Close_slope10', 'Close_slope25', 'SMA_10_Lag10_max', 'SMA_10_Lag200_min', 'Williams_%R_5', 'Zscore_10', 'Zscore_5', 'Zscore_50']
Run 3/5 | Train: 2021-03-22 → 2026-02-04 | Test: 2026-02-09 → 2026-02-09 | Train_n=1225 | Test_n=1 | (PI Years: 1.5 - Feats: 8)
71 | 8 | All Cols: ['CCI_14', 'Close_slope10', 'Close_slope25', 'SMA_10_Lag200_min', 'SMA_50_Lag150_min', 'Williams_%R_5', 'Zscore_10', 

,run,model,test_days,pred,acc,test_n,test_pos_n,test_neg_n,test_pos_frac,test_neg_frac,...,train_end,test_start,test_end,train_years,n_features,pi_size,pi_handling,min_feats,features,horizon
0,1,xgboost-1,1,0.80,0.0,1,0,1,0.0,1.0,...,2026-02-06,2026-02-11,2026-02-11,5,8,1.5,include_new,8,ma_lag-experimental_slope,2
1,2,xgboost-1,1,0.30,1.0,1,0,1,0.0,1.0,...,2026-02-05,2026-02-10,2026-02-10,5,8,1.5,include_new,8,ma_lag-experimental_slope,2
2,3,xgboost-1,1,0.00,1.0,1,0,1,0.0,1.0,...,2026-02-04,2026-02-09,2026-02-09,5,8,1.5,include_new,8,ma_lag-experimental_slope,2
3,4,xgboost-1,1,0.05,0.0,1,1,0,1.0,0.0,...,2026-02-03,2026-02-06,2026-02-06,5,8,1.5,include_new,8,ma_lag-experimental_slope,2
4,5,xgboost-1,1,0.70,1.0,1,1,0,1.0,0.0,...,2026-02-02,2026-02-05,2026-02-05,5,8,1.5,include_new,8,ma_lag-experimental_slope,2


- Train through most recent actualized data (Done)
- Append all actuals to raw files
- Recalc performance
- Predict on new horizon
- Show prediction

# Archived

In [158]:
def normalize_csvs():
    
    for r in returns:
        
        past_performance = pd.read_csv(f"h{r}_baseline.csv") # Match prior cell saved as file name horizon_2_baseline_new
        """
        past_performance["features"] = (
        past_performance["feature_set"]
            .str.replace("_ba$", "", regex=True)
            .str.replace("-baseline$", "", regex=True)
            .str.replace("-pr$", "-past_return", regex=True)
            .str.replace("past_ret_cols", "past_return", regex=False)
        )
        """

        past_performance["pi_handling"] = (
        past_performance["pi_size"]
        .str.replace("1.5-r", "run_separately", regex=False)
        .str.replace("1.5", "include_new", regex=False)
        )

        past_performance["pi_size"] = 1.5
        past_performance.to_csv(f"h{r}_baseline.csv", index=False)